<a href="https://colab.research.google.com/github/RedDawe/GAN-MNIST/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 40.6MB/s 
     |████████████████████████████████| 450kB 49.1MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
from google.colab.patches import cv2_imshow

In [3]:
import tensorflow as tf
import numpy as np
tf.executing_eagerly()

True

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

print(x_train.shape, x_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [5]:
x = np.concatenate([x_train, x_test], axis=0)
x.shape

(70000, 28, 28, 1)

In [6]:
generator = tf.keras.Sequential([
  tf.keras.layers.Input([100]),
  tf.keras.layers.Dense(128*7*7),
  tf.keras.layers.Reshape([7, 7, 128]),
  tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=4, strides=2, activation='relu', padding='same'),
  tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=4, strides=2, activation='relu', padding='same'),
  tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=7, strides=1, activation='sigmoid', padding='same')
])

generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       262272    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 1)         6273      
Total params: 1,164,289
Trainable params: 1,164,289
Non-trainable params: 0
_________________________________________________________________


In [7]:
discriminator = tf.keras.Sequential([
  tf.keras.layers.Input([28, 28, 1]),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu', padding='same'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu', padding='same'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3137      
Total params: 40,705
Trainable params: 40,705
Non-trainable params: 0
__________________________________________________

In [8]:
generator.trainable = True
discriminator.trainable = False

gan = tf.keras.Sequential([
  tf.keras.Input([100]),
  generator,
  discriminator,
  tf.keras.layers.Lambda(lambda a: 1-a)
])

gan.summary()
gan.compile(tf.keras.optimizers.Adam(0.0002, beta_1=0.5), 'binary_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 28, 28, 1)         1164289   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 40705     
_________________________________________________________________
lambda (Lambda)              (None, 1)                 0         
Total params: 1,204,994
Trainable params: 1,164,289
Non-trainable params: 40,705
_________________________________________________________________


In [0]:
discriminator.trainable = True
discriminator.compile(tf.keras.optimizers.Adam(0.0002, beta_1=0.5), 'binary_crossentropy', metrics=['accuracy'])

In [0]:
batch_size = 8192

for step in range(30):
  print('STEP:', step)

  for batch in range(0, x.shape[0], batch_size//2):
    #gan.fit(x=np.random.randn(batch_size, 49), y=np.zeros([batch_size]), batch_size=batch_size, epochs=1,  verbose=0)
    g_loss = gan.train_on_batch(x=np.random.randn(batch_size, 100), y=np.zeros([batch_size]))

    fake_x = generator.predict(np.random.randn(batch_size//2, 100))
    real_x = x[batch:batch+batch_size//2]

    X = np.concatenate([fake_x, real_x], axis=0)
    Y = np.concatenate([np.zeros(batch_size//2), np.ones(real_x.shape[0])], axis=0)

    #discriminator.fit(X, Y, batch_size=batch_size,  epochs=1, verbose=0)
    d_loss = discriminator.train_on_batch(X, Y)

    print(g_loss, d_loss)

STEP: 0
[0.7066437, 0.33947754] [0.6879035, 0.5701904]
[0.71860003, 0.21105957] [0.6828331, 0.62023926]
[0.72902477, 0.12573242] [0.6774532, 0.6694336]
[0.73901546, 0.072265625] [0.6729189, 0.7072754]
[0.7492916, 0.03881836] [0.6678343, 0.7375488]
[0.7581933, 0.018920898] [0.661793, 0.77001953]
[0.7664584, 0.008056641] [0.6582842, 0.7800293]
[0.7739797, 0.0028076172] [0.6537162, 0.81188965]
[0.7807586, 0.0018310547] [0.6489178, 0.84106445]
[0.7839346, 0.0013427734] [0.64561284, 0.86901855]


In [0]:
generated_img = generator.predict(np.random.randn(1, 100))[0]*255
cv2_imshow(cv2.resize(generated_img, (256, 256)))